# QBM Comparison Exact vs. Annealer

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext autotime

from joblib import Parallel, delayed

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import neptune.new as neptune
from numba import njit
from scipy.constants import k as k_B, h as h_P

k_B /= h_P * 1e9
matplotlib.rcParams.update({"font.size": 14})

from qbm.models import BQRBM
from qbm.plotting import plot_qq
from qbm.utils import (
    binarize_df,
    convert_bin_list_to_str,
    get_binarization_params,
    get_project_dir,
    get_rng,
    kl_divergence,
    load_artifact,
    lr_exp_decay,
    prepare_training_data,
    save_artifact,
    unbinarize_df,
    compute_stats_over_dfs,
)
from qbm.utils.exact_qbm import get_pauli_kron, compute_H, compute_rho

# configure directories
project_dir = get_project_dir()
qpu_params = {"region": "na-west-1", "solver": "Advantage_system4.1"}
artifact_dir = project_dir / f"artifacts/qbm/8x4/{qpu_params['solver']}"
plot_dir = project_dir / f"results/plots/qbm/8x4/{qpu_params['solver']}"
if not artifact_dir.exists():
    artifact_dir.mkdir(parents=True)
if not plot_dir.exists():
    plot_dir.mkdir(parents=True)

# load anneal schedule data
if qpu_params["solver"] == "Advantage_system4.1":
    csv_name = "09-1263A-A_Advantage_system4_1_annealing_schedule.csv"
elif qpu_params["solver"] == "Advantage_system5.1":
    csv_name = "09-1265A-A_Advantage_system5_1_annealing_schedule.csv"
df_anneal = pd.read_csv(
    project_dir / f"data/anneal_schedules/csv/{csv_name}", index_col="s",
)
if 0.5 not in df_anneal.index:
    df_anneal.loc[0.5] = (df_anneal.loc[0.499] + df_anneal.loc[0.501]) / 2
df_anneal.sort_index(inplace=True)

time: 1.59 s (started: 2022-03-23 23:32:49 +01:00)


## Train Data Creation

In [2]:
seed = 42
n_visible = 8
n_hidden = 4
n_qubits = n_visible + n_hidden

rng = get_rng(seed)
n_samples = 1500
α = 2 / 3
N_1 = rng.normal(-2, 1, int(round(n_samples * α, 0)))
N_2 = rng.normal(3, 1, int(round(n_samples * (1 - α), 0)))
x = np.concatenate((N_1, N_2))

df = pd.DataFrame.from_dict({"x": x})
binarization_params = get_binarization_params(df, n_bits=n_visible)
df_binarized = binarize_df(df, binarization_params)
X_train = prepare_training_data(df_binarized)["X_train"]
X_train.shape

(1500, 8)

time: 10 ms (started: 2022-03-23 23:32:51 +01:00)


In [3]:
data = unbinarize_df(df_binarized, binarization_params)

n_bins = 32
fig, ax = plt.subplots(dpi=144, figsize=(10, 6))
_, bins, _ = ax.hist(data, bins=n_bins, density=True, label="Data")
ax.grid(alpha=0.7)
plt.tight_layout()
plt.savefig(plot_dir / "hist_data.png")

<Figure size 1440x864 with 1 Axes>

time: 240 ms (started: 2022-03-23 23:32:51 +01:00)


In [4]:
def callback(model, sample_state_vectors):
    X_train = model.X_train
    n_visible = model.X_train.shape[1]
    train_states = (
        model._eigen_to_binary(model.X_train) * 2.0 ** np.arange(n_visible - 1, -1, -1)
    ).sum(axis=1)
    sample_states = (
        model._eigen_to_binary(sample_state_vectors[:, :n_visible])
        * 2.0 ** np.arange(n_visible - 1, -1, -1)
    ).sum(axis=1)

    dkl = kl_divergence(train_states, sample_states, n_bins=32)

    return {"value": dkl, "print": f"D_KL = {dkl:.3f}"}

time: 652 µs (started: 2022-03-23 23:32:51 +01:00)


## Model Analysis (Exact)

In [5]:
s_freeze = 1.0
train_model = False
if train_model:
    # model params
    embedding = None
    beta_initial = 1
    exact_params = {"beta": 0.5}

    # training params
    n_epochs = 100
    n_samples = 10_000
    learning_rate = 0.1
    mini_batch_size = 10
    decay_epoch = 50
    decay_period = 10
    epochs = np.arange(1, n_epochs + 1)
    learning_rates = learning_rate * lr_exp_decay(epochs, decay_epoch=50, period=10)
    learning_rates_beta = learning_rate * lr_exp_decay(
        epochs, decay_epoch=50, period=20
    )

    anneal_params = {
        "s": s_freeze,
        "A": df_anneal.loc[s_freeze, "A(s) (GHz)"],
        "B": df_anneal.loc[s_freeze, "B(s) (GHz)"],
    }
    # model init
    model_exact = BQRBM(
        X_train=X_train,
        n_hidden=n_hidden,
        embedding=embedding,
        anneal_params=anneal_params,
        beta_initial=beta_initial,
        exact_params=exact_params,
    )

    # model train and save
    model_exact.train(
        n_epochs=n_epochs,
        n_samples=n_samples,
        learning_rate=learning_rates,
        learning_rate_beta=learning_rates_beta,
        mini_batch_size=mini_batch_size,
        callback=callback,
    )
    model_exact.save(artifact_dir / f"models/exact/model-s_freeze={s_freeze}.pkl")
    model_exact_metrics = {
        "A": model_exact.A,
        "B": model_exact.B,
        "a": model_exact.a,
        "b": model_exact.b,
        "W": model_exact.W,
        "beta": model_exact.beta,
        "embedding": model_exact.embedding,
        "anneal_params": model_exact.anneal_params,
        "exact_params": model_exact.exact_params,
        "beta_history": model_exact.beta_history,
        "callback_outputs": [x for x in model_exact.callback_outputs],
    }
    save_artifact(
        model_exact_metrics,
        artifact_dir / f"models/exact/model-s_freeze={s_freeze}-attributes.pkl",
    )
else:
    model_exact = BQRBM.load(
        artifact_dir / f"models/exact/model-s_freeze={s_freeze}.pkl"
    )

time: 11.7 ms (started: 2022-03-23 23:32:51 +01:00)


In [6]:
samples_exact = model_exact.sample(10 ** 6, binary=True)
state_vectors_exact = samples_exact["state_vectors"][:, :n_visible]
df_samples_exact = pd.DataFrame(
    {"x": [convert_bin_list_to_str(x) for x in state_vectors_exact]}
)
samples_exact = unbinarize_df(df_samples_exact, binarization_params)
data = unbinarize_df(df_binarized, binarization_params)

n_bins = 32
fig, ax = plt.subplots(dpi=144, figsize=(10, 6))
_, bins, _ = ax.hist(data, bins=n_bins, density=True, label="Data")
ax.hist(
    samples_exact,
    bins=bins,
    density=True,
    alpha=0.5,
    color="tab:orange",
    label="Model (Simulation)",
)
ax.grid(alpha=0.7)
ax.legend()
plt.tight_layout()
plt.savefig(plot_dir / "hist_comparison_exact.png")

<Figure size 1440x864 with 1 Axes>

time: 4.89 s (started: 2022-03-23 23:32:51 +01:00)


In [7]:
dkls = [x["value"] for x in model_exact.callback_outputs]
epochs = np.arange(1, len(dkls) + 1)
fig, ax = plt.subplots(1, 2, figsize=(10, 5), dpi=300)
ax[0].plot(epochs, dkls, linewidth=1.8)
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel(r"$D_{KL}(p_{data} \ || \ p_{model})$")
ax[0].set_yticks(np.arange(0, 22.5, 2.5) / 100)
ax[1].set_yticks(np.arange(40, 110, 10))
ax[0].set_ylim((0, 0.2))
ax[1].set_ylim((40, 100))
ax[0].grid(alpha=0.7)

ax[1].plot(
    range(len(model_exact.beta_history)),
    1 / k_B / np.array(model_exact.beta_history) * 1000,
    linewidth=1.8,
)
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel(r"$\hat{T}$ [mK]")
ax[1].axhline(1 / k_B / 0.5 * 1000, color="k", linestyle="--", label=r"$T_{effective}$", linewidth=1.8)
ax[1].grid(alpha=0.7)
ax[1].legend(loc="lower right")
plt.tight_layout()
plt.savefig(plot_dir / "train_results_exact.png")

<Figure size 3000x1500 with 2 Axes>

time: 756 ms (started: 2022-03-23 23:32:56 +01:00)


## Model Analysis (Annealer)

In [8]:
# set the model params
train_model = False
embedding_id = 10
s_pause = 0.55
s_freeze = 1.0

# load embedding
embedding = load_artifact(artifact_dir / f"embeddings/embedding_{embedding_id:02}.json")
embedding = {int(k): v for k, v in embedding.items()}

# set anneal schedule
t_r = 20
α_quench = 2
t_pause = round(s_pause * t_r, 3)
Δ_quench = round((1 - s_pause) / α_quench, 3)
if s_pause == 1:
    anneal_schedule = [(0, 0), (t_pause, s_pause)]
else:
    anneal_schedule = [
        (0, 0),
        (t_pause, s_pause),
        (round(t_pause + Δ_quench, 3), 1),
    ]

# set model name and path
model_name = (
    f"model-s_pause={s_pause:.2f}-s_freeze={s_freeze:.2f}-embedding_{embedding_id:02}"
)
model_path = artifact_dir / f"models/annealer/{model_name}.pkl"
if train_model:
    # model_annealer params
    beta_initial = 0.5
    exact_params = None

    # training params
    n_epochs = 100
    n_samples = 10_000
    learning_rate = 0.1
    mini_batch_size = 10
    epochs = np.arange(1, n_epochs + 1)
    learning_rates = learning_rate * lr_exp_decay(epochs, decay_epoch=50, period=10)
    learning_rates_beta = learning_rate * lr_exp_decay(
        epochs, decay_epoch=50, period=20
    )

    # set the anneal params
    anneal_params = {
        "s": s_freeze,
        "A": df_anneal.loc[s_freeze, "A(s) (GHz)"],
        "B": df_anneal.loc[s_freeze, "B(s) (GHz)"],
        "schedule": anneal_schedule,
    }

    # skip if model already exists
    if model_path.exists():
        raise Exception("Model already exists")

    # model init
    model_annealer = BQRBM(
        X_train=X_train,
        n_hidden=n_hidden,
        embedding=embedding,
        anneal_params=anneal_params,
        beta_initial=beta_initial,
        exact_params=exact_params,
        qpu_params=qpu_params,
    )

    # model train and save
    model_annealer.train(
        n_epochs=n_epochs,
        n_samples=n_samples,
        learning_rate=learning_rates,
        learning_rate_beta=learning_rates_beta,
        mini_batch_size=mini_batch_size,
        callback=callback,
    )
    model_annealer.save(model_path)

    # save attributes as dict in case of error loading old pickled object
    model_annealer_attributes = {
        "A": model_annealer.A,
        "B": model_annealer.B,
        "a": model_annealer.a,
        "b": model_annealer.b,
        "W": model_annealer.W,
        "beta": model_annealer.beta,
        "embedding": model_annealer.embedding,
        "qpu_params": model_annealer.qpu_params,
        "anneal_params": model_annealer.anneal_params,
        "exact_params": model_annealer.exact_params,
        "beta_history": model_annealer.beta_history,
        "callback_outputs": [x for x in model_annealer.callback_outputs],
    }
    save_artifact(
        model_annealer_attributes,
        artifact_dir / f"models/annealer/{model_name}-attributes.pkl",
    )
else:
    model_annealer = BQRBM.load(model_path)

time: 2.7 s (started: 2022-03-23 23:32:57 +01:00)


In [9]:
# generate samples with the annealer trained model
model = BQRBM.load(model_path)
ensemble_size = 10
samples_dir = artifact_dir / "samples"
samples_annealer = {}
for sample_id in range(1, ensemble_size + 1):
    samples_path = samples_dir / model_name / f"samples_{sample_id:02}.pkl"
    if samples_path.exists():
        samples_annealer[sample_id] = load_artifact(samples_path)
    else:
        samples_annealer[sample_id] = model_annealer.sample(
            n_samples=10_000, binary=True
        )
        save_artifact(samples_annealer[sample_id], samples_path)

time: 1.64 s (started: 2022-03-23 23:33:00 +01:00)


In [10]:
# train a classical RBM for comparison purposes
from sklearn.neural_network import BernoulliRBM

train_model = False
if train_model:
    rng = get_rng(42)
    model_RBM = BernoulliRBM(
        n_components=n_hidden,
        learning_rate=1e-3,
        batch_size=10,
        n_iter=10_000,
        random_state=rng,
    )
    model_RBM.fit(X_train)

    samples_RBM = rng.choice([0, 1], (10 ** 6, n_visible))
    for i in range(1_000):
        samples_RBM = model_RBM.gibbs(samples_RBM)

    save_artifact(model_RBM, artifact_dir / "RBM/model.pkl")
    save_artifact(samples_RBM, artifact_dir / "RBM/samples.pkl")
else:
    model_RBM = load_artifact(artifact_dir / "RBM/model.pkl")
    samples_RBM = load_artifact(artifact_dir / "RBM/samples.pkl")

time: 220 ms (started: 2022-03-23 23:33:01 +01:00)


In [11]:
train_states = (X_train * 2.0 ** np.arange(n_visible - 1, -1, -1)).sum(axis=1)
dkls_annealer_ = {}
for sample_id, sample_set in samples_annealer.items():
    sample_states = (
        sample_set.record.sample[:, :n_visible]
        * 2.0 ** np.arange(n_visible - 1, -1, -1)
    ).sum(axis=1)

    dkls_annealer_[sample_id] = kl_divergence(train_states, sample_states, n_bins=32)

dkls_annealer_stats = {
    "mean": np.mean(list(dkls_annealer_.values())),
    "std": np.std(list(dkls_annealer_.values())),
    "min": np.min(list(dkls_annealer_.values())),
    "max": np.min(list(dkls_annealer_.values())),
}

samples_exact_test = model_exact.sample(n_samples=10 ** 6, binary=True)[
    "state_vectors"
][:, :n_visible]
samples_exact_test = (samples_exact_test * 2.0 ** np.arange(n_visible - 1, -1, -1)).sum(
    axis=1
)
dkl_exact_test = kl_divergence(train_states, samples_exact_test, n_bins=32)

samples_RBM_test = (samples_RBM * 2.0 ** np.arange(n_visible - 1, -1, -1)).sum(axis=1)
dkl_RBM_test = kl_divergence(train_states, samples_RBM_test, n_bins=32)

time: 271 ms (started: 2022-03-23 23:33:02 +01:00)


In [12]:
state_vectors = samples_annealer[1].record.sample[:, :n_visible]
df_samples = pd.DataFrame({"x": [convert_bin_list_to_str(x) for x in state_vectors]})
samples_annealer_df = unbinarize_df(df_samples, binarization_params)
samples_annealer_df
data = unbinarize_df(df_binarized, binarization_params)

time: 51.3 ms (started: 2022-03-23 23:33:02 +01:00)


In [13]:
n_bins = 32
fig, ax = plt.subplots(dpi=144, figsize=(10, 6))
_, bins, _ = ax.hist(data, bins=n_bins, density=True, label="Data")
ax.hist(
    samples_annealer_df,
    bins=bins,
    density=True,
    alpha=0.5,
    color="tab:orange",
    label="BQRBM Advantage 4.1",
)
ax.grid(alpha=0.7)
ax.legend()
plt.tight_layout()
plt.savefig(plot_dir / "hist_comparison_annealer.png")

<Figure size 1440x864 with 1 Axes>

time: 314 ms (started: 2022-03-23 23:33:02 +01:00)


In [14]:
qq_params = {
    "xlims": (-6.5, 6.5),
    "ylims": (-6.5, 6.5),
    "xticks": np.arange(-6, 7, 2),
    "yticks": np.arange(-6, 7, 2),
}

fig, axs = plt.subplots(1, 2, dpi=144, figsize=(10, 5))
plot_qq(
    axs[0],
    data.to_numpy(),
    samples_exact.to_numpy()[: len(data)],
    title=r"Simulation",
    params=qq_params,
)
plot_qq(
    axs[1],
    data.to_numpy(),
    samples_annealer_df.to_numpy()[1000 : 1000 + len(data)],
    title="Advantage 4.1",
    params=qq_params,
)
plt.tight_layout()
plt.savefig(plot_dir / "qq_comparison.png")

<Figure size 1440x720 with 2 Axes>

time: 350 ms (started: 2022-03-23 23:33:02 +01:00)


In [15]:
def smooth(x, k=1):
    x_out = np.zeros(len(x))
    for i in range(len(x)):
        x_out[i] = np.mean(x[i - min((k - 1, i)) : i + 1])
    return x_out


markers = ["o", "^", "v", "<", ">", "s", "p", "*", "P", "X"]
colors = [
    "tab:blue",
    "tab:orange",
    "tab:green",
    "tab:red",
    "tab:purple",
    "tab:brown",
    "tab:pink",
    "tab:gray",
    "tab:olive",
    "tab:cyan",
]
fig, ax = plt.subplots(1, 2, figsize=(10, 5), dpi=300)
dkls_exact = [d["value"] for d in model_exact.callback_outputs]
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel(r"$D_{KL}(p_{data} \ || \ p_{model})$")
ax[0].set_yticks(np.arange(0, 22.5, 2.5) / 100)
ax[1].set_yticks(np.arange(80, 110, 2))
ax[0].set_ylim((0, 0.2))
# ax[1].set_ylim((80, 120))
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel(r"$\hat{T}$ [mK]")
# ax[1].set_ylim((0.45, 0.55))
i = 0
dkls_annealer = [d["value"] for d in model_annealer.callback_outputs]

epochs = np.arange(1, len(dkls_exact) + 1)
ax[0].plot(
    epochs, smooth(dkls_annealer), label=fr"BQRBM Advantage 4.1", color=colors[i], linewidth=1.8
)
ax[0].plot(
    epochs, smooth(dkls_exact), label=r"BQRBM Simulation", color="k", linestyle="--", linewidth=1.8
)
ax[0].axhline(
    dkl_RBM_test,
    label="RBM (Final Result)",
    linestyle="dotted",
    c="tab:gray",
    linewidth=2.5,
)

ax[1].plot(
    range(len(model_annealer.beta_history)),
    smooth(1 / k_B / np.array(model_annealer.beta_history) * 1000),
    color=colors[i],
    linewidth=1.8
)

ax[0].grid(alpha=0.7)
ax[0].legend()
ax[1].grid(alpha=0.7)
plt.tight_layout()
plt.savefig(plot_dir / "train_results_annealer.png")

<Figure size 3000x1500 with 2 Axes>

time: 673 ms (started: 2022-03-23 23:33:03 +01:00)


## Exact Heatmap

In [16]:
@njit(boundscheck=True)
def kl_divergence_exact(
    p_exact,
    E_exact,
    E_samples,
    counts_samples,
    n_bins=32,
    prob_sum_tol=1e-6,
    ϵ_smooth=1e-6,
):
    """
    Computes the KL divergence of the theory w.r.t. the samples, i.e., 
    D_KL(p_exact || p_samples).
    
    :param p_exact: Exact computed probability vector, i.e., the diagonal of ρ.
    :param E_exact: Exact computed energy vector, i.e., the diagonal of H.
    :param E_samples: Energies of the samples.
    :param n_bins: Number of bins to compute over.
    :param prob_sum_tol: The tolerance for the probabilities to sum up to approx 1.
    :param ϵ_smooth: Smoothing parameter for the samples distribution.
    
    :returns: D_KL(p_exact || p_samples).
    """
    p = np.zeros(n_bins)
    q = np.zeros(n_bins)

    # compute the bin edges
    buffer = np.abs(E_exact).max() * 1e-15
    bin_edges = np.linspace(E_exact.min() - buffer, E_exact.max() + buffer, n_bins + 1)

    # check that bin edges include all possible E values
    assert bin_edges.min() <= E_exact.min()
    assert bin_edges.max() >= E_exact.max()

    # bin the probabilities
    sum_counts = counts_samples.sum()
    for i, (a, b) in enumerate(zip(bin_edges[:-1], bin_edges[1:])):
        if i < n_bins - 1:
            p[i] = p_exact[np.logical_and(E_exact >= a, E_exact < b)].sum()
            q[i] = (
                counts_samples[np.logical_and(E_samples >= a, E_samples < b)].sum()
                / sum_counts
            )
        else:
            p[i] = p_exact[E_exact >= a].sum()
            q[i] = counts_samples[E_samples >= a].sum() / sum_counts

    # smoothing of sample data
    smooth_mask = np.logical_and(p > 0, q == 0)
    not_smooth_mask = np.logical_not(smooth_mask)
    q[smooth_mask] = p[smooth_mask] * ϵ_smooth
    q[not_smooth_mask] -= q[smooth_mask].sum() / not_smooth_mask.sum()

    # check that p and q sum up to approx 1
    assert np.abs(p.sum() - 1) < prob_sum_tol
    assert np.abs(q.sum() - 1) < prob_sum_tol

    # take intersection of supports to avoid div zero errors
    support_intersection = np.logical_and(p > 0, q > 0)
    p = p[support_intersection]
    q = q[support_intersection]

    return (p * np.log(p / q)).sum()


@njit(boundscheck=True)
def get_state_energies(states, E_exact):
    """
    Returns the (quantum + classical) energies of the provided states corresponding
    to the provided exact calculated energies.
    
    :param states: Array of states. Must be a value in 0, 1, ..., 2 ** n_qubits - 1.
    :param E_exact: Array of exact computed energies, corresponds to the diagonal of H.
    
    :returns: Array where entry i is the energy of states[i].
    """
    E_samples = np.zeros(len(states))
    for i, state in enumerate(states):
        E_samples[i] = E_exact[state]

    return E_samples


def convert_spin_vector_to_state_number(spins):
    """
    Converts the spins vector (e.g. all values ±1) to an integer corresponding to the state.
    For example, the spin vector [1, 1, 1, 1] corresponds to the state |0000⟩ which is the
    0th state. The spin vector [-1, -1, -1, -1] corresponds to the state |1111⟩ which is the
    15th state.
    
    :param spins: Vector of spin values (±1).
    
    :returns: Integer corresponding to the state. 
    """
    bit_vector = ((1 - spins) / 2).astype(np.int64)

    return (bit_vector * 2 ** np.arange(len(bit_vector) - 1, -1, -1)).sum()


def kl_divergence_df(exact_data, samples):
    """
    Compares each exact computed data distribution against the provided samples instance.
    
    :param exact_data: Dictionary with keys of the form (s, T) with s being the relative
        anneal time at which H and ρ were computed, and T being the effective temperature.
        Values are of the form {"E": [...], "p": [...]}
    :param samples: Instance of Ocean SDK SampleSet.
    
    :returns: Dataframe of KL divergences, with T values as index and s values as columns.
    """
    # convert spin vectors to state numbers
    states = np.array(
        [convert_spin_vector_to_state_number(x) for x in samples.record.sample]
    )

    dkl = {}
    for s, T in exact_data.keys():
        p_exact = exact_data[(s, T)]["p"]
        E_exact = exact_data[(s, T)]["E"]
        E_samples = get_state_energies(states, E_exact)

        dkl[int(T * 1000), s] = kl_divergence_(
            p_exact, E_exact, E_samples, samples.record.num_occurrences
        )

    return pd.Series(dkl)

time: 21.5 ms (started: 2022-03-23 23:33:03 +01:00)


In [17]:
compute_distributions = False
compute_dkls = False

T_values = np.arange(0, 152, 2) / 1000
T_values[0] = 1e-6
s_values = np.arange(0, 101, 1) / 100
pauli_kron = get_pauli_kron(n_visible, n_hidden)

h = model_annealer.h
J = model_annealer.J

distributions = {}
if compute_distributions:
    for s_value in s_values:
        A = df_anneal.loc[s_value, "A(s) (GHz)"]
        B = df_anneal.loc[s_value, "B(s) (GHz)"]
        for T_value in T_values:
            beta = 1 / k_B / T_value
            H = compute_H(h, J, A, B, n_qubits, pauli_kron)
            rho = compute_rho(H, beta, diagonal=(A == 0))
            distributions[s_value, T_value] = {
                "E": H.diagonal().copy(),
                "p": rho.diagonal().copy(),
            }

    save_artifact(
        distributions,
        artifact_dir / f"exact_distributions/distributions-s_pause={s_pause:.2f}.pkl",
    )
else:
    distributions = load_artifact(
        artifact_dir / f"exact_distributions/distributions-s_pause={s_pause:.2f}.pkl"
    )


def compute_dkls_sample(distribution, sample_energies, sample_id):
    dkl = kl_divergence_exact(
        distribution["p"],
        distribution["E"],
        E_samples=sample_energies[sample_id - 1],
        counts_samples=np.ones(len(sample_energies[sample_id - 1])),
    )

    return dkl


dkls = {}
sample_ids = range(1, 11)
if compute_dkls:
    for (s_value, T_value), distribution in distributions.items():
        sample_energies = []
        for sample_id in sample_ids:
            sample = samples_annealer[sample_id]
            states = np.array(
                [
                    convert_spin_vector_to_state_number(x)
                    for x in model_annealer._binary_to_eigen(sample.record.sample)
                ]
            )
            sample_energies.append(get_state_energies(states, distribution["E"]))

        dkls_sT = Parallel(n_jobs=6)(
            delayed(compute_dkls_sample)(distribution, sample_energies, sample_id)
            for sample_id in sample_ids
        )

        dkls[s_value, T_value] = np.mean(dkls_sT)

    save_artifact(
        dkls, artifact_dir / f"exact_distributions/dkls-s_pause={s_pause:.2f}.pkl"
    )
else:
    dkls = load_artifact(
        artifact_dir / f"exact_distributions/dkls-s_pause={s_pause:.2f}.pkl"
    )

time: 272 ms (started: 2022-03-23 23:33:03 +01:00)


In [18]:
import seaborn as sns
from matplotlib.patches import Rectangle

dkl = pd.DataFrame.from_dict(dkls, orient="index")
dkl.index = pd.MultiIndex.from_tuples(dkl.index)
dkl = dkl.unstack(level=0)[0]

xticks = np.arange(len(s_values))[::20]
xticklabels = s_values[::20]
yticks = np.arange(len(T_values))[::10]
yticklabels = np.round(T_values[::10] * 1000).astype(np.int64)

fig, ax = plt.subplots(figsize=(8, 6), dpi=300)

cbar_kws = {"label": r"$D_{KL}(p_{theory} \ || \ p_{samples})$", "ticks": [0, 0.05, 0.1, 0.15, 0.2]}
cmap = sns.color_palette("rocket_r", as_cmap=True)
sns.heatmap(dkl, ax=ax, cmap=cmap, vmin=0, vmax=0.2, cbar_kws=cbar_kws)
ax.invert_yaxis()
ax.set_xlabel(r"$s$")
ax.set_ylabel(r"$T$ [mK]")
ax.set_xticks(xticks + 0.5)
ax.set_xticklabels(xticklabels, rotation=0)
ax.set_yticks(yticks + 0.5)
ax.set_yticklabels(yticklabels, rotation=0)
plt.tight_layout()

T_model = 1 / model_annealer.beta / k_B
x_min = 1 * 100
y_min = T_model / 2 * 1000
ax.scatter(
    [x_min],
    [y_min],
    marker="x",
    s=64,
    c="tab:blue",
    label=fr"$\hat{{T}} = {round(T_model * 1000, 1)}$ mK",
)
T_optimal = dkl.index[np.argmin(dkl[1.0])]
x_min = 1 * 100
y_min = T_optimal / 2 * 1000
# ax.scatter(
#     [x_min],
#     [y_min],
#     marker="x",
#     c="tab:red",
#     label=fr"$T_{{effective}} = {round(T_optimal * 1000, 1)} \pm 2.0$ mK",
# )
s_mins = np.round(np.arange(50, 101, 1) / 100, 2)
T_mins = []
B_mins = []
βB_mins = []
for s in s_mins:
    i = np.argmin(dkl.loc[:, s])
    T_mins.append(dkl.index[i])
    B_mins.append(df_anneal.loc[s, "B(s) (GHz)"])
    βB_mins.append(B_mins[-1] / k_B / T_mins[-1])
T_mins = 1 / k_B / np.mean(βB_mins) * np.array(B_mins)
ax.plot(
    s_mins * 100,
    T_mins / 2 * 1000,
    linestyle="--",
    color="k",
    label=fr"$B(s) / T = {np.mean(βB_mins) * k_B:.1f}}}$ GHz/K",
)
T_DW = 15.4 / 2
ax.axhline(T_DW, c="w", label=r"$T_{DW} = 15.4 \pm 0.1$ mK")


ax.legend(loc="upper left")

plt.savefig(plot_dir / "effective_temperature.png")


def round_(x, nearest, n):
    return round(nearest * round(x / nearest), n)


print(f"Optimal T = {df_anneal.loc[1, 'B(s) (GHz)'] / 88.6 * 1000:.1f} mK")

Optimal T = 95.7 mK


<Figure size 2400x1800 with 2 Axes>

time: 660 ms (started: 2022-03-23 23:33:04 +01:00)
